## helper functions for map.ipynb

note that all directory paths returned contain the trailing slash, e.g., './data/adcp-files/'

In [ ]:
import os
import glob

import pandas as pd
import numpy as np

import re

from datetime import datetime
import jdcal

import ipywidgets as widgets
import ipyleaflet

In [ ]:
def get_meta_fldr():
    """
    :return: string path to directory containing ADCP metadata files
    """
    return ('./data/adcp-metadata-files/')

def get_fig_fldr():
    """
    :return: string path to directory containing ADCP echo backscatter figures
    """
    return ('./data/adcp-backscatter-figs/')

def get_cr_fldrs():
    """
    get cruise folders by finding all folders in the ADCP data directory that end in the pattern (2xLetter)(4xNumber)
    e.g., './data/adcp-files/lg0501/'
    
    :return: list of paths of all cruise folders
    """
    cr_fldrs = glob.glob(get_meta_fldr()+'*') # get all folders within the ADCP data directory
    regexp = re.compile(r'\b\w{2}\d{4}\b') # regexp to get all folders that match the pattern (2xLetter)(4xNumber)
    cr_fldrs = [s+'/' for s in cr_fldrs if regexp.search(s)] # append a slash
    return(cr_fldrs)

def get_instr_fldrs(instr):
    """
    get list of cruise/instrument/ folders
    
    :param instr: instrument code; will always be 'nb150' for the narrowband 150 kHz ADCP in this repo
    :return: list of paths of all cruise/instrument/ folders
    """
    cr_fldrs = get_cr_fldrs()
    return ([cr+instr+'/' for cr in cr_fldrs])

def get_cr_instr_fldr(cruise,instr):
    """
    get list of cruise/instrument/ folders by appending the instrument string folder ("/instr/") to each cruise folder path

    :param cruise: (string) cruise code, e.g., 'lg0501'
    :param instr: instrument code; will always be 'nb150' for the narrowband 150 kHz ADCP in this repo
    :return: list of paths of all cruise/instrument folders
    """
    return get_meta_fldr() + cruise + '/' + instr + '/'

In [ ]:
def get_cruises():
    """
    get list of cruise codes (example cruise code: 'lg0501')
    
    :return: list of cruise code strings
    """
    cr_fldrs = get_cr_fldrs()
    return([fl.split('/')[-2] for fl in cr_fldrs])

def pal_cruise_from_year(y):
    """
    get cruise code from year, e.g., 2005 --> 'lg0501'
    
    :param year: year (numeric or string)
    :return: cruise code string (e.g., 'lg0501')
    """
    if isinstance(y, float): # if float, convert to int before converting to string to avoid e.g. '2005.0' 
        y = int(y)        
    y = str(y)
    return "lg"+y[-2:]+"01"

def year_from_cruise(cr):
    """
    get year from cruise code, e.g., 'lg0501' --> 2005
    
    :param cr: cruise code string (e.g., 'lg0501') 
    :return: year (int)
    """
    pattern = r'\d{2}'
    year = re.findall(pattern, cr)
    year = year[0]
    year = int(year)
    year = year+1900 if year > 50 else year + 2000
    return(year)

In [ ]:
def get_CLSTHr(hr):
    """
    convert hour in UTC to CLST (Chile Summer Time, the local time of PAL LTER cruises)
    
    :param hr: hour in UTC
    :return: hour in CLST
    """
    hr = hr-3
    if hr>=0:
        return(hr)
    else:
        return(24+hr)

In [ ]:
def get_base_map():
    """
    get Antarctic Base Map (EPSG:3031) zoomed on Antarctic Peninsula
    
    :return: ipyleaflet.Map - Antarctic Base Map  
    """
    #Base Map
    ESRI_3031 = dict(
        name='EPSG:3031',
        custom=True,
        proj4def="""+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1
            +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs""",
        origin=[-3.06361E7, 3.0636099999999993E7],
        resolutions=[
            67733.46880027094,
            33866.73440013547,
            16933.367200067736,
            8466.683600033868,
            4233.341800016934,
            2116.670900008467,
            1058.3354500042335,
            529.1677250021168,
            264.5838625010584,
            132.2919312505292,
            66.1459656252646,
            33.0729828126323,
            16.53649140631615,
            8.268245703158074
        ],
        bounds=[
            [-4524583.19363305,-4524449.487765655],
            [4524449.4877656475,4524583.193633042]
        ],
    )

    tile_layer = ipyleaflet.TileLayer(url='https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Antarctic_Basemap/MapServer/tile/{z}/{y}/{x}',max_zoom=18,max_native_zoom=8)
    spsLayout = widgets.Layout(width='1000px', height='900px')

    base_map = ipyleaflet.Map(center=(-66, -63),
            zoom=6,
            layout=spsLayout,
            basemap= tile_layer,
            crs=ESRI_3031)
    base_map.add_control(ipyleaflet.FullScreenControl())
    return(base_map)